In [1]:
import numpy as np
import os
import pandas as pd
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import random
import seaborn as sns
%matplotlib inline
from scipy.cluster.hierarchy import fcluster
import h5py
from io import StringIO
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import itertools
import xlsxwriter as Excel
from joblib import Parallel, delayed
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import datasets
from sklearn.datasets import make_blobs
from numba import jit
import networkx as nx

In [2]:
data_path = "samples"
#converts to df to binary dataframe, 0 representing that the previous value was 0, and 1 indicating a nonzero value
def df_to_binary(df_expression):    
    temp = df_expression.copy()
    num_rows = len(temp)
    for c in range(len(df_expression.columns)):
        nonzero_inds = df_expression.iloc[:,c].to_numpy().nonzero()[0]
        temp_col = np.zeros(num_rows)
        temp_col[nonzero_inds] = 1
        temp.iloc[:,c] = temp_col
    return(temp)



#INPUTS:
#  file_list: list of csv filenames containing expression data. Will concatenate these matrices together and compute distances
def preprocess_samples(file_list):
    sample_source = []
    # Merge all of the files into one dataframe
    df_expression = pd.read_csv(os.path.join(data_path,file_list[0]),index_col = 0,sep="\t")
    sample_source.extend(np.repeat(file_list[0],len(df_expression.columns)))
    for filename in file_list[1:]:
        temp = pd.read_csv(os.path.join(data_path,filename),index_col = 0, sep = "\t")
        sample_source.extend(np.repeat(filename,len(temp.columns)))
        df_expression = df_expression.merge(temp,right_index=True,left_index=True)
    
    df_expression.columns = pd.Series(df_expression.columns)
    df_expression.index = pd.Series(df_expression.index)
    
    return(df_expression,sample_source)    

def hierarchical_clustering(df,title,method="average",metric="euclidean"):
    clustering = shc.linkage(df, method=method, metric=metric)
    plt.figure(figsize=(25, 20))  
    plt.title(title)  

    dend = shc.dendrogram(clustering) 
    print("Cophenet = " + str(cophenet(clustering, pdist(df))[0]))
    return(clustering)

#INPUTS: 
#  h5filepath: path where the full matrix is stored with meta and expression data
#  subsamples: a pandas DataFrame that must contain a column (foreign key) called SampleGeoAccession
#OUTPUT
#  merged dataframe of length(subsamples) merged with relevant meta/Sample values for cluster analysis
def h5_sample_meta_lookup(h5filepath,subsamples):
    f = h5py.File(h5filepath, 'r')
    full_data = pd.DataFrame({"Description":f[("meta/Sample_description")], 
                          "Characteristics":f[("meta/Sample_characteristics_ch1")],
                          "SampleGeoAccession":f[("meta/Sample_geo_accession")],
                          "Series ID":f[("meta/Sample_series_id")],
                          "Molecule":f["meta/Sample_molecule_ch1"],
                          "Source Name":f["meta/Sample_title"]})
    f.close()
    full_data["SampleGeoAccession"] = full_data["SampleGeoAccession"].apply(lambda x : x.decode("utf-8"))
    cluster_meta = subsamples.merge(full_data,on="SampleGeoAccession",how="left")
    cluster_meta["Sample_Label"] = pd.Series(sample_source).str.replace(".csv","").str.replace("_expression"," ")
    cluster_meta["Series ID"] = cluster_meta["Series ID"].str.decode("utf-8").str.replace("Xx","").str.replace("xX","")
    cluster_meta["Molecule"] = cluster_meta["Molecule"].str.decode("utf-8")
    cluster_meta["Source Name"] = cluster_meta["Source Name"].str.decode("utf-8")
    cluster_meta["Description"] = cluster_meta["Description"].str.decode("utf-8").str.replace("Xx","").str.replace("xX","")
    cluster_meta["Characteristics"] = cluster_meta["Characteristics"].str.decode("utf-8").str.replace("Xx","").str.replace("xX","")
    return(cluster_meta)

def purity_to_excel(cluster_metadata_metrics, H0, outfile):
    workbook = Excel.Workbook("tables/" +outfile +".xlsx")
    worksheet = workbook.add_worksheet()
    cluster_format = workbook.add_format({'bold': True, 'align':'center'})
    center = workbook.add_format({'align': 'center'})

    # Add a format. Light red fill with dark red text.
    format1 = workbook.add_format({'bg_color': '#FFC7CE',
                                   'font_color': '#9C0006'})

    # Add a format. Green fill with dark green text.
    format2 = workbook.add_format({'bg_color': '#C6EFCE',
                                   'font_color': '#006100'})

    worksheet.write('A1', "Cluster ID",cluster_format)
    worksheet.write('B1', "Cluster Size",center)
    worksheet.write('C1', "Dominant Molecule",center)
    worksheet.write('D1', "Molecule Purity",center)
    worksheet.write('E1', "Dominant Series",center)
    worksheet.write('F1', "Series Purity",center)
    worksheet.write('G1', "Dominant Label",center)
    worksheet.write('H1', "Label Purity",center)
    worksheet.write('I1', "Null Hypothesis Value",center)

    worksheet.set_column(1, 7, 15)

    num_keys = len(list(cluster_metadata_metrics.keys()))

    j = 1
    for i in np.sort(list(cluster_metadata_metrics.keys())):
        worksheet.write(j,0,i,center)
        worksheet.write(j,1,cluster_metadata_metrics[i]["Size"],center)
        worksheet.write(j,2,list(cluster_metadata_metrics[i]["Molecule Purity"].keys())[0],center)
        worksheet.write(j,3,list(cluster_metadata_metrics[i]["Molecule Purity"].values())[0],center)
        worksheet.write(j,4,list(cluster_metadata_metrics[i]["Series Purity"].keys())[0],center)
        worksheet.write(j,5,list(cluster_metadata_metrics[i]["Series Purity"].values())[0],center)
        worksheet.write(j,6,list(cluster_metadata_metrics[i]["Label Purity"].keys())[0],center)
        worksheet.write(j,7,list(cluster_metadata_metrics[i]["Label Purity"].values())[0],center)
        worksheet.write(j,8,H0[list(cluster_metadata_metrics[i]["Label Purity"].keys())[0]],center)
        
        

        # Write a conditional format over a range.
        worksheet.conditional_format('H'+str(j+1), {'type': 'cell',
                                             'criteria': '>=',
                                             'value': H0[list(cluster_metadata_metrics[i]["Label Purity"].keys())[0]],
                                             'format': format2})

        # Write another conditional format over the same range.
        worksheet.conditional_format('H'+str(j+1), {'type': 'cell',
                                             'criteria': '<',
                                             'value': H0[list(cluster_metadata_metrics[i]["Label Purity"].keys())[0]],
                                             'format': format1})

        j += 1

    workbook.close()


def get_purity(cluster_assignments, df_expression_trans):
    cluster_dict = {"SampleGeoAccession":[],"ClusterAssignment":[]}
    cluster_dict["SampleGeoAccession"] = df_expression_trans.index
    cluster_dict["ClusterAssignment"] = cluster_assignments
    cluster_info = pd.DataFrame(cluster_dict)
    cluster_metadata = h5_sample_meta_lookup("D:\\548project\\human_matrix.h5",cluster_info).sort_values("ClusterAssignment")
    
    #hypothesized values of expected sample frequencies assuming no separation (based on observed proportion of sample labels)
    H0 = {}
    for samp in cluster_metadata.Sample_Label:
        H0[samp] = len(cluster_metadata[cluster_metadata.Sample_Label == samp])/len(cluster_metadata)
    
    cluster_metadata_metrics = {}
    for c in range(1,k+1):
        cluster_metadata_metrics[c] = {"Size":len(cluster_metadata.loc[cluster_metadata.ClusterAssignment == c]), 
                                       "Molecule Purity":{},"Series Purity":{},"Label Purity":{}}
        series_counts = cluster_metadata.groupby('ClusterAssignment')["Series ID"].value_counts()
        molecule_counts = cluster_metadata.groupby('ClusterAssignment')["Molecule"].value_counts()
        label_counts = cluster_metadata.groupby('ClusterAssignment')["Sample_Label"].value_counts()

        series_purity_key = pd.Series(series_counts[c][series_counts[c] == series_counts[c].max()]).index[0]
        series_purity_val = series_counts[c].max()/float(series_counts[c].sum())
        cluster_metadata_metrics[c]["Series Purity"][series_purity_key] = round(series_purity_val,3)

        molecule_purity_key = molecule_counts[c][molecule_counts[c] == molecule_counts[c].max()].index[0]
        molecule_purity_val = molecule_counts[c].max()/float(molecule_counts[c].sum())
        cluster_metadata_metrics[c]["Molecule Purity"][molecule_purity_key] = round(molecule_purity_val,3)

        label_purity_key = label_counts[c][label_counts[c] == label_counts[c].max()].index[0]
        label_purity_val = label_counts[c].max()/float(label_counts[c].sum())
        cluster_metadata_metrics[c]["Label Purity"][label_purity_key] = round(label_purity_val,3)
    purity_to_excel(cluster_metadata_metrics,H0,cluster_title+" purity")

In [3]:
file_list = ["ALPHA_expression.csv","beta_expression.csv","PANC1_expression.csv"]
df_expression_raw,sample_source = preprocess_samples(file_list)

In [4]:
df_expression_norm = (df_expression_raw - df_expression_raw.mean())/(df_expression_raw.std())
method="average"
metric="euclidean"
#df_expression_trans = np.transpose(df_expression_norm)
df_expression_trans = np.transpose(df_expression_raw)
#df_expression_trans = np.transpose(df_to_binary(df_expression_trans))
dataTitle="Normalized"
cluster_title = dataTitle +" Data (Filtered) Liver vs. HEPG2 cell "+method+"+"+metric
df_expression_trans = np.transpose(df_expression_norm)
genes = list(df_expression_trans.columns)
samples = list(df_expression_trans.index)

In [6]:
def duplicates(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

def eHKnn(distance,K):
    #Create the nodenext matrix which stores the neighbor in the neighborhood
    #of size K. This needs to be supplemented by the nodes linked by the Minimum
    #Spanning Tree. See pre-print
    N=distance.shape[0]
    nodenext=[]
    Rank=np.argsort(distance)
    Rank=Rank[:,:K+1]
    r_ = np.zeros( (N,K), dtype=int)
    for i in range(N):
        rank = list(Rank[i])
        rank.remove(i)
        r_[i] = rank[:K]
    Rank =  r_
    for i in range(N):
        e=[]
        for j in Rank[i]:
            if i in Rank[j]:
                e.append(j)
        nodenext.append(e)
    return nodenext

def eHK(link,nodenext):
    #Extended Hoshen-Kopelman implementation. See pre-print

    N=len(nodenext)
    nodel=10*N*np.ones(N,dtype=int)
    label_counter=0
    nodelp=np.array([],dtype=int)
    for i in range(N):
        if (np.array(link[i])==0).all():
            nodel[i]=label_counter
            nodelp=np.append(nodelp,label_counter)
            label_counter+=1
        else:
            idx = duplicates(link[i],1) #where links are
            t=[nodel[ nodenext[i][j] ] for j in idx]
            t=np.array(t)
            if (t==10*N).all(): # all unlabeled?
                nodel[i]=label_counter
                nodelp=np.append(nodelp,label_counter)
                label_counter+=1
            else:
                w=[]
                for index in range(len(t)):
                    if t[index]!=10*N:
                        w.append(index)
                idx_ = np.array([nodenext[i][j] for j in idx])
                z = nodelp[nodel[ idx_[w] ] ]
                min_ = np.amin(z)

                nodel[i] = min_
                a = nodel[ idx_[w] ]
                nodelp[a]=min_

    # sequentialize part 1: re-order nodelp
    for y in range(len(nodelp)):
        n = y
        while (nodelp[n]<n):
            n=nodelp[n]
        nodelp[y]=n
    # sequentialize part 2: get rid of the gaps
    un = np.unique(nodelp)
    for i in range(len(un)-1):
        while un[i+1]-un[i] !=1:
            idx = np.where(nodelp==un[i+1])[0]
            nodelp[idx] -= 1
            un = np.unique(nodelp)

    # rename the labels with their root
    for i in range( len(nodelp) ):
        nodel[nodel==i]=nodelp[i]

    return nodel



def cHKlons(nodenext,G):
    # This function serves to perform the consensus final cluster solution
    # using the Spin Spin correlation matrix G
    link=[]
    N=G.shape[0]
    for i in range(N):
        neighbors=nodenext[i]
        e=[]
        for j in neighbors:
            if (G[i,j]>0.5):
                e.append(1)
            else:
                e.append(0)
        idx=np.argmax(G[i,neighbors])
        e[idx]=1
        link.append(e)
    # make sure the neighbor with the highest correlation is linked both ways
    for i in range(N):
        idx = duplicates(link[i],1)
        for f in idx:
            X = nodenext[i][f]
            Y = duplicates(nodenext[X],i)
            Y = Y[0]
            link[X][Y] = 1
    return link


def kron(i, j):
    #kronecker delta
    if i == j:
        return 1
    else:
        return 0

def twopc(S,cij):
    #two point connectedness
    classes=np.unique(S)
    for label in classes:
        neighbors=duplicates(S,label)
        for node in neighbors:
            cij[node,neighbors]+=1
    return cij

def Hs(S, J, nodenext):
   # Hamiltonian Energy
    E=0
    N = len(S)
    for i in range(N):
        for j in nodenext[i]:
            E += J[i, j]*(1-kron(S[i], S[j]))
    return E/N

def magnetization(S, q):
    N=len(S)
    nmax = np.amax(np.bincount(S))
    return (q*nmax-N)/((q-1)*N)

def runz(S,f,mcmc,nodenext,J,t,q,K):
    np.random.seed(0)
    def flip(S, q):
        # Flip clusters labels after Monte Carlo steps
        c = np.unique(S)  # find unique labels
        new_c = np.random.randint(0, q, len(c))  #gen new spins for clusters
        conv = dict(zip(c, new_c))  # use dic to assign new spins to clusters
        return np.vectorize(conv.get)(S)

    def eHKlons(nodenext,T,J,S):
        # Create link matrix, which stores the edges activation status
        link=[]
        N = len(nodenext)
        for i in range(N):
            e=[]
            for j in nodenext[i]:
                if (1-np.exp(-J[i,j]*kron(S[i],S[j])/T)>np.random.uniform() ):
                    e.append(1)
                else:
                    e.append(0)
            link.append(e)
        return link

    N=len(S)
    forget=int(f*mcmc)
    m=np.zeros(mcmc)
    cij=np.zeros((N,N))
    
    for i in range(forget):
        # the number of SW steps that allow the system to reach thermal eq
        S1=S
        E1=Hs(S1,J,nodenext)
        LinkSnode = eHKlons(nodenext,t,J,S)
        S = eHK(LinkSnode,nodenext)
        S = flip(S, q)
        E2 = Hs(S,J,nodenext)
        if E2 >= E1:
            if np.exp(- E2 / t) < np.random.uniform() :
                S=S1

    for i in range(mcmc):
        #  Actual SW steps we keep
        S1=S
        E1=Hs(S1,J,nodenext)
        LinkSnode = eHKlons(nodenext,t,J,S)
        S = eHK(LinkSnode,nodenext)
        S = flip(S, q)
        E2 = Hs(S,J,nodenext)
        if E2 >= E1:
            if np.exp(- E2 / t) < np.random.uniform() :
                S=S1
                E2 = E1

        cij=twopc(S,cij)
        m[i]=magnetization(S,q)
    # Compute thermodynamic averages here 
    mbar = np.average(m)  #  Average magnetization
    su = N*np.var(m)/t #  Magnetic Susceptibility
    return su, mbar, cij, S


In [7]:
# data is coming from "dataframe.name".to_numpy()
# nc is the cluster number
# x is the iteration times. Example uses 60.
def SPC(data,nc,x):
    
    # number of genes in this data
    N = data.shape[0]
    T = np.linspace(1e-6,.3,num=x,endpoint=True)
    
    distance = euclidean_distances(data)
    
    # number of mcmc steps
    mcmc = 200
    
    # Determine the Graph, and its minimal spanning tree
    Tree=nx.minimum_spanning_tree(nx.from_numpy_matrix(distance))
    # Determine the neighborhood and add the Minimal Spanning Tree edges on top of it
    
    #Create the nodenext matrix which stores the neighbor in the neighborhood
    # of size K. This needs to be supplemented by the nodes linked by the Minimum
    # Spanning Tree. 
    K=10
    nodenext = eHKnn(distance, K)
    
    #  Add the edges in the minimal spanning tree not in nodenext
    mst_edges = list( Tree.edges())
    for i in mst_edges:
        node = i[0]
        if i[1] not in nodenext[node]:
            nodenext[node].append(i[1])
            nodenext[node] = sorted(nodenext[node])
            nodenext[i[1]].append(node)
            nodenext[i[1]] =  sorted(nodenext[i[1]])
    
    # need average number of neighbors khat, and the local length scale a
    khat = 0
    a = 0
    alpha = 4
    
    for i in nodenext:
        khat+= len(i)
        khat = khat / N
        
    for i in range(N):
        a+=sum(distance[i,nodenext[i]])
        a = alpha * a / (khat*N)
    
    # Interaction Strength
    n = 2
    J = (1 / khat) * np.exp(-( (n-1)/n ) * ( distance / a)**n)
    
    # How many mcmc steps are forgotten for every temperature t
    f_=0.5

    # The initial spin configuration S_0 for all temperatures
    S=np.ones(N, dtype=int)
    
    #SPC runs sequentially but every temperatures are ran in parallel
    results = Parallel(n_jobs=5)(delayed( runz )(S,f_,mcmc,nodenext,J,T[y],nc,K) for y in range(x))
    
    return results

In [10]:
data1 = df_expression_trans.iloc[1:10,:].to_numpy()
a = SPC(data1,20,60)

ValueError: cannot copy sequence with size 8 to array axis with dimension 10

In [25]:
a[59][3]

array([18, 18, 18, 18, 18, 18, 17, 18, 18, 18, 18, 17, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 17, 17, 18, 18, 18, 18, 17, 18, 18, 18, 18,
       17, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17

example which works well

In [30]:
project = 'blobs'
blob = datasets.make_blobs(n_samples=500,
                           cluster_std=[0.25,0.5,1],
                            random_state=0, n_features=500,shuffle=True)
data = blob[0]
N = data.shape[0]
T=np.linspace(1e-6,.3,num=60,endpoint=True)
K = 10
q = 20
alpha = 4
distance = euclidean_distances(data)


mcmc = 200

k = len(T)

Tree=nx.minimum_spanning_tree(nx.from_numpy_matrix(distance))

nodenext = eHKnn(distance, K)
mst_edges = list( Tree.edges())

for i in mst_edges:
    node = i[0]
    if i[1] not in nodenext[node]:
        nodenext[node].append(i[1])
        nodenext[node] = sorted(nodenext[node])
        nodenext[i[1]].append(node)
        nodenext[i[1]] =  sorted(nodenext[i[1]])

khat = 0
for i in nodenext:
    khat+= len(i)
khat = khat / N

a = 0
for i in range(N):
    a+=sum(distance[i,nodenext[i]])
a = alpha * a / (khat*N)

n = 2
J = (1 / khat) * np.exp(-( (n-1)/n ) * ( distance / a)**n)


f_=0.5


S=np.ones(N, dtype=int)

results = Parallel(n_jobs=5)(delayed( runz )(S,f_,mcmc,nodenext,J,T[y],q,K) for y in range(k))

In [31]:
results[59][3]

array([ 4, 17,  7, 10, 10, 13, 16,  0,  8, 14, 10,  2, 12, 11,  1,  6,  5,
       14,  2,  1, 16,  4,  8,  8, 16,  0,  8,  6, 13,  1,  9, 15,  5, 17,
        6,  6, 18, 11, 18,  6, 13,  7,  4,  8,  8,  6,  8, 11,  3, 19, 13,
       10,  2, 19, 10,  8, 19,  4,  4,  0, 16, 10, 19,  8, 12, 11,  4,  7,
       11,  6, 17,  6, 13, 15, 17, 11,  0,  7, 13,  0, 14, 12, 12,  1, 12,
       15, 16,  7, 15,  8,  3, 16,  2, 13, 12,  7, 11, 11,  5,  9,  1, 15,
        5,  0, 17,  7, 16,  3, 13, 16, 11,  0,  9,  8, 18,  4,  3,  9, 17,
       16,  6,  6, 17, 17, 13,  1,  9,  6,  7, 18, 18,  1,  3, 11, 13, 16,
        8,  4, 12,  9,  2, 12,  9,  9, 12,  4,  0, 17, 18,  6, 12, 16,  1,
       17,  6, 13,  4,  9,  1,  2,  2, 11,  8, 16, 17,  4, 17,  1,  8,  1,
       16,  9, 16, 17, 13, 16,  9, 16, 19,  9,  4, 12, 18,  1,  1, 16, 13,
       13, 11,  4,  4,  6, 12, 17,  5,  6, 17, 12, 18, 18,  2, 17,  1, 11,
        9,  6, 16, 11, 15,  1,  5,  8,  1, 12, 14, 11, 18,  8, 11, 11, 17,
        2, 14, 14,  2, 14